In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"

Running local


In [2]:
import os
import numpy as np

import datetime
from datetime import timedelta

import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from Jaguas_DataLoader_rainless import SoundscapeData

import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

def find_wav_filenames( path_to_dir, suffix=".wav" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

In [ ]:
files_Ais = find_csv_filenames(f"{root}/Zamuro/AI_Caract")
file_rain = f"{root}/Zamuro/Complementary_Files/Result_rain_merged.xlsx"
files_udas = f"{root}/Zamuro/Complementary_Files/UDAS Pasivo_20221001_Zamuro.xlsx"
files_covers = f"{root}/Zamuro/Complementary_Files/ReyZamuro_softLabels.csv"
files_recorders = os.listdir("/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/")
path_recorders = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/"

In [ ]:
UDAS = pd.read_excel(f"{files_udas}", sheet_name="Template")
covers = pd.read_csv(f"{files_covers}")
# AIs = pd.read_csv(files_Ais)
habitat = ["bosque"]* 2 + ["pastizal"] + ["bosque"] + ["borde"] * 2 + ["bosque"] +  ["parche"] *2 + ["bosque"]*4 + ["borde"]*11 + ["bosque"] * 9 + ["parche"]*2 + ["pastizal"]*4 + ["parche"]*2 + ["sabana"]*10 + ["borde"] + ["sabana"] + ["pastizal"] + ["sabana"] + ["bosque"]*3 + ["pastizal"]*2 + ["potrero"] * 2 + ["bosque"] + ["borde"]* 5 +["bosque"] * 9 + ["sabana"]* 2 + ["pastizal"] + ["parche"]
habitat = [
    "bosque", "bosque", "pastizal", "borde", "borde", "borde", "bosque", "sabana", "bosque", "borde",
    "bosque", "bosque", "bosque", "bosque", "bosque", "borde", "borde", "pastizal", "potrero", "sabana",
    "borde", "bosque", "bosque", "sabana", "borde", "bosque", "borde", "bosque", "bosque", "bosque", "borde",
    "potrero", "bosque", "bosque", "sabana", "bosque", "bosque", "sabana", "parche", "bosque", "pastizal",
    "bosque", "bosque", "borde", "bosque", "borde", "borde", "bosque", "parche", "bosque", "bosque", "sabana",
    "bosque", "pastizal", "pastizal", "sabana", "sabana", "borde", "bosque", "sabana", "bosque", "sabana",
    "bosque", "sabana", "borde", "borde", "borde", "borde", "pastizal", "bosque", "pastizal", "pastizal",
    "bosque", "bosque", "bosque", "bosque", "sabana", "pastizal", "parche", "borde", "sabana", "sabana", "parche",
    "parche", "sabana", "borde", "sabana", "parche", "bosque", "sabana", "parche", "bosque", "sabana", "borde",
    "borde", "sabana", "sabana", "borde", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque"
]

keep_cols = ["field_number_PR", "longitud_IG", "latitude_IG", "day_init_TI", "day_end_TI", "samp_rate_RE"]

In [ ]:
UDAS = UDAS[keep_cols]
UDAS["habitat"] = habitat
UDAS.info()
covers.rename(columns={"field_numb":"field_number_PR", "1":"pasture", "2": "savanna", "3": "forest"}, inplace=True)
a=UDAS
b=covers
merged_df = a.merge(b, on='field_number_PR', how='left')
result_df = merged_df[merged_df['pasture'].notna()]
result_df.reset_index(inplace=True)
result_df.drop("index", axis=1, inplace=True)
result_df["cover"] = result_df[["pasture", "savanna", "forest"]].idxmax(axis=1)
result_df.to_csv(f"{root}//Zamuro/Complementary_Files/zamuro_recorders.csv")

In [ ]:
audios = pd.DataFrame(columns=result_df.columns)
total_files=0
for i in range(len(result_df)):
    aux = pd.DataFrame(columns=result_df.columns)
    path = root_path+"/"+result_df.loc[i, "field_number_PR"]
    files = os.listdir(path)
    total_files+=len(files)
    aux["Filename"] = files
    name=[i.split('.', 1)[0] for i in files]
    aux["name_FI"] = name
    for col in aux.columns[0:-2]:
        aux[col] = result_df.loc[i,col]
    audios = pd.concat([audios, aux])
audios.to_csv(f"{root}/Zamuro/Complementary_Files/zamuro_audios.csv")
audios.reset_index(inplace=True)

In [ ]:
rain = pd.read_excel(file_rain)
rain = rain[["field_number_PR", "name_FI","rain_FI"]]
audios["rain_FI"] = "-"
count = 0
for i in range(len(audios)):
    if i%10000 == 0:
        print(i)
    try:
        audio = audios.loc[i]
        audios.loc[i, "rain_FI"] = rain.loc[rain["field_number_PR"] == audio["field_number_PR"]].loc[rain["name_FI"] == audio["Filename"]]["rain_FI"].values[0]
    except:
        count+=1
audios.to_csv(f"{root}//Zamuro/Complementary_Files/zamuro_audios.csv")

In [9]:
audios_zamuro = pd.read_csv(f"{root}//Zamuro/Complementary_Files/zamuro_audios.csv")
audios_zamuro.drop(columns=["Unnamed: 0"], inplace=True)
set(audios_zamuro["field_number_PR"])

{'RZUA01',
 'RZUA02',
 'RZUA03b',
 'RZUA04',
 'RZUA05',
 'RZUA06',
 'RZUA07',
 'RZUA08',
 'RZUA09',
 'RZUA10',
 'RZUB01',
 'RZUB02',
 'RZUB03',
 'RZUB04',
 'RZUB05',
 'RZUB06',
 'RZUB07',
 'RZUB08',
 'RZUB09',
 'RZUB10',
 'RZUB11',
 'RZUC01',
 'RZUC02',
 'RZUC03',
 'RZUC05',
 'RZUC06',
 'RZUC07',
 'RZUC08',
 'RZUC09',
 'RZUC10',
 'RZUC11',
 'RZUC12',
 'RZUD01M',
 'RZUD02',
 'RZUD03',
 'RZUD04',
 'RZUD05',
 'RZUD06',
 'RZUD07',
 'RZUD08',
 'RZUD09',
 'RZUD10',
 'RZUD11',
 'RZUD12',
 'RZUD13',
 'RZUE01',
 'RZUE02',
 'RZUE03',
 'RZUE04',
 'RZUE05',
 'RZUE06',
 'RZUE07',
 'RZUE08',
 'RZUE09',
 'RZUE10',
 'RZUE11',
 'RZUE12',
 'RZUE13',
 'RZUF02',
 'RZUF03',
 'RZUF04',
 'RZUF05',
 'RZUF06',
 'RZUF07',
 'RZUF08',
 'RZUF09',
 'RZUF10',
 'RZUF11',
 'RZUF12',
 'RZUF13',
 'RZUG02',
 'RZUG03',
 'RZUG04',
 'RZUG05',
 'RZUG06',
 'RZUG07',
 'RZUG08',
 'RZUG09',
 'RZUG10',
 'RZUG11',
 'RZUG12',
 'RZUG13',
 'RZUH02',
 'RZUH03',
 'RZUH04',
 'RZUH05',
 'RZUH06',
 'RZUH07',
 'RZUH08',
 'RZUH09',
 'RZUH10

In [16]:
audios_zamuro

,index,field_number_PR,longitud_IG,latitude_IG,day_init_TI,day_end_TI,samp_rate_RE,habitat,pasture,savanna,forest,cover,Filename,name_FI,rain_FI
0,0,RZUA01,-73.411659,3.555860,3,14,192000,bosque,1.000000,0.000000,0.00000,pasture,20220906_063000.WAV,20220906_063000,NO
1,1,RZUA01,-73.411659,3.555860,3,14,192000,bosque,1.000000,0.000000,0.00000,pasture,20220909_023000.WAV,20220909_023000,YES
2,2,RZUA01,-73.411659,3.555860,3,14,192000,bosque,1.000000,0.000000,0.00000,pasture,20220903_104500.WAV,20220903_104500,YES
3,3,RZUA01,-73.411659,3.555860,3,14,192000,bosque,1.000000,0.000000,0.00000,pasture,20220903_110000.WAV,20220903_110000,NO
4,4,RZUA01,-73.411659,3.555860,3,14,192000,bosque,1.000000,0.000000,0.00000,pasture,20220903_111500.WAV,20220903_111500,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78311,767,RZUD01M,-73.400768,3.555762,3,11,192000,bosque,0.786941,0.000008,0.21305,pasture,20220911_074500.WAV,20220911_074500,NO
78312,768,RZUD01M,-73.400768,3.555762,3,11,192000,bosque,0.786941,0.000008,0.21305,pasture,20220911_080000.WAV,20220911_080000,NO
78313,769,RZUD01M,-73.400768,3.555762,3,11,192000,bosque,0.786941,0.000008,0.21305,pasture,20220911_081500.WAV,20220911_081500,NO
78314,770,RZUD01M,-73.400768,3.555762,3,11,192000,bosque,0.786941,0.000008,0.21305,pasture,20220911_083000.WAV,20220911_083000,NO


In [106]:
temporal_csv

,global_index,field_number_PR,longitud_IG,latitude_IG,day_init_TI,day_end_TI,samp_rate_RE,habitat,pasture,savanna,forest,cover,Filename,name_FI,rain_FI
3654,0,RZUA03b,-73.41152,3.548784,8,14,192000,pastizal,0.705108,0.294892,0.0,pasture,20220911_044500.WAV,20220911_044500,NO
3655,1,RZUA03b,-73.41152,3.548784,8,14,192000,pastizal,0.705108,0.294892,0.0,pasture,20220908_090000.WAV,20220908_090000,YES
3656,2,RZUA03b,-73.41152,3.548784,8,14,192000,pastizal,0.705108,0.294892,0.0,pasture,20220908_091500.WAV,20220908_091500,NO
3657,3,RZUA03b,-73.41152,3.548784,8,14,192000,pastizal,0.705108,0.294892,0.0,pasture,20220908_093000.WAV,20220908_093000,NO
3658,4,RZUA03b,-73.41152,3.548784,8,14,192000,pastizal,0.705108,0.294892,0.0,pasture,20220908_094500.WAV,20220908_094500,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4223,569,RZUA03b,-73.41152,3.548784,8,14,192000,pastizal,0.705108,0.294892,0.0,pasture,20220913_074500.WAV,20220913_074500,NO
4224,570,RZUA03b,-73.41152,3.548784,8,14,192000,pastizal,0.705108,0.294892,0.0,pasture,20220913_120000.WAV,20220913_120000,NO
4225,571,RZUA03b,-73.41152,3.548784,8,14,192000,pastizal,0.705108,0.294892,0.0,pasture,20220913_161500.WAV,20220913_161500,NO
4226,572,RZUA03b,-73.41152,3.548784,8,14,192000,pastizal,0.705108,0.294892,0.0,pasture,20220913_203000.WAV,20220913_203000,NO


In [113]:
lista = list(set(audios_zamuro["field_number_PR"]))
lista.sort()
for csv in lista:
    temporal_csv = audios_zamuro.loc[audios_zamuro["field_number_PR"] == csv]
    temporal_csv.drop(columns=["index"],inplace=True)
    temporal_csv.to_csv(f"{root}//Zamuro/Complementary_Files/{csv}.csv",
                        index=True, index_label="global_index")

/tmp/ipykernel_17785/3568608739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_csv.drop(columns=["index"],inplace=True)
/tmp/ipykernel_17785/3568608739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_csv.drop(columns=["index"],inplace=True)
/tmp/ipykernel_17785/3568608739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_csv.drop(columns=["index"],inplace=True)
/tmp/ipykernel_17785/3568608739.py:5:

/tmp/ipykernel_17785/3568608739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_csv.drop(columns=["index"],inplace=True)
/tmp/ipykernel_17785/3568608739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_csv.drop(columns=["index"],inplace=True)
/tmp/ipykernel_17785/3568608739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_csv.drop(columns=["index"],inplace=True)
/tmp/ipykernel_17785/3568608739.py:5:

/tmp/ipykernel_17785/3568608739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_csv.drop(columns=["index"],inplace=True)
/tmp/ipykernel_17785/3568608739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_csv.drop(columns=["index"],inplace=True)
/tmp/ipykernel_17785/3568608739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_csv.drop(columns=["index"],inplace=True)
/tmp/ipykernel_17785/3568608739.py:5:

In [121]:
proof = pd.read_csv(f"{root}//Zamuro/Complementary_Files/{lista[93]}.csv")
len(lista)

94